In [ ]:
import numpy as np
from numpy.linalg import eigh, eig
import pandas as pd
import plotly.express as px
from plotly import graph_objects as go
from scipy.optimize import curve_fit

import importlib

import PCA
importlib.reload(PCA)


from jinja2 import Template

output=r"../slides/index-A.html"
input = r"../slides/index-to-build.html"

In [ ]:
map = PCA.Map('../data/GA29/mapa2_laser532nm_100_mocy_1akumulacja_5s_CRR_step0p3um.txt', begin=1250, end=1450)

In [ ]:
map.plotWeights(False, True).show()

In [ ]:
map.plotBaseVectors(7).show()

In [ ]:
## Map A
t_matrix = np.zeros((7,7))

draw = False
# for i in range(6):
#     t_matrix[0][i] = np.average(map_a.base_vectors[i], weights=((map_a.raman_shift < 1310) | (map_a.raman_shift > 1420)) )
    # t_matrix[0][i] = map_a.draw_curve(line, [np.average(map_a.base_vectors[i], weights=((map_a.raman_shift < 1310) | (map_a.raman_shift > 1420)) )], 1200, 1500, base_vector = i, draw = True, bounds = 'inf').show()

t_matrix[0][0] = map_a.fit_curve(lorentz, (1392, 2.5, -17 * 10**5, -7000), 1380, 1400, 2, base_vector = 0, draw = draw)

t_matrix[1][0] = map_a.fit_curve(lorentz, (1345, 14,  -5 * 10**5, -10000), 1330, 1360, 2, base_vector = 0, draw = draw)
t_matrix[1][3] = map_a.fit_curve(lorentz, (1345, 16,  3.2 * 10**5, -4000), 1330, 1360, 2, base_vector = 3, draw = draw)
t_matrix[1][5] = map_a.fit_curve(lorentz, (1345, 14,  -5 * 10**5, -10000), 1330, 1360, 2, base_vector = 5, draw = draw)
t_matrix[1][6] = map_a.fit_curve(lorentz, (1345, 16,  3.2 * 10**5, -4000), 1330, 1360, 2, base_vector = 6, draw = draw)

t_matrix[2][1] = map_a.fit_curve(lorentz, (1369, 10,  0.8 * 10**5,  1000), 1355, 1380, 2, base_vector = 1, draw = draw)
t_matrix[2][2] = map_a.fit_curve(lorentz, (1369, 10, -2.4 * 10**5, -1400), 1355, 1380, 2, base_vector = 2, draw = draw)


t_matrix[3][1] = map_a.fit_curve(lorentz_diff1, (1392, 2.5, -17 * 10**5, -7000), 1385, 1400, 2, base_vector = 1, draw = draw, bounds = 'inf')
t_matrix[3][2] = map_a.fit_curve(lorentz_diff1, (1392, 2.5, -17 * 10**5, -7000), 1385, 1400, 2, base_vector = 2, draw = draw, bounds = 'inf')
t_matrix[3][3] = map_a.fit_curve(lorentz_diff1, (1392, 2.5, -17 * 10**5, -7000), 1385, 1400, 2, base_vector = 3, draw = draw, bounds = 'inf')

t_matrix[4][4] = map_a.fit_curve(lorentz_diff2, [ 1392, 3, 10.8e+04, -1400], 1380, 1400, 2, base_vector = 4, draw = draw, bounds = 'inf')

t_matrix[5][5] = map_a.fit_curve(lorentz_diff3, [ 1392, 3, 10.8e+04, -1400], 1380, 1400, 2, base_vector = 5, draw = draw, bounds = 'inf')
t_matrix[5][6] = map_a.fit_curve(lorentz_diff3, [ 1392, 3, 10.8e+04, -1400], 1380, 1400, 2, base_vector = 5, draw = draw, bounds = 'inf')

t_matrix[6][5] =  t_matrix[5][5]
t_matrix[6][6] = -t_matrix[5][6]

t_matrix

In [ ]:
map_a.draw_curve(lorentz_diff2, [ 1.39113772e+03, 3e+00, 10.79995135e+04, -1.38837064e+03], 1380, 1400, base_vector = 4, draw = draw, bounds = 'inf').write_html('test.html')


In [ ]:
refit_matrix = np.array([
    [1, 0, 0, 0, 0, 0, 0],
    [-.0, 1, 0, -17/50 + .15, -.3, .25, 0],
    [.05, 0, 1, .1, -.34, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1],
])

refit_matrix2 = np.array([
    [1, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, -1.8, 0, 0, 0, 1, 0],
    [0, -2, 0, -.05, -.22, 0, 1],
])

refit_matrix3 = np.array([
    [1, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, -0.1, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1],
])

tmp = np.dot(np.linalg.inv(t_matrix), np.dot(refit_matrix2.T, refit_matrix.T))

better_t = tmp.T
better_t = np.dot(refit_matrix3, better_t)

new_base_vectors = np.dot(better_t, map_a.base_vectors[:7])
new_vectors = np.dot(map_a.vectors[:7].T, np.linalg.inv(better_t)).T

# px.line(x = map_a.raman_shift, y = new_base_vectors[0]).show()
px.imshow(np.reshape(new_vectors[0], map_a.shape)).show()
# px.line(x = map_a.raman_shift, y = new_base_vectors[1]).show()
px.imshow(np.reshape(new_vectors[1], map_a.shape)).show()
# px.line(x = map_a.raman_shift, y = new_base_vectors[2]).show()
px.imshow(np.reshape(new_vectors[2], map_a.shape)).show()
# px.line(x = map_a.raman_shift, y = new_base_vectors[3]).show()
# px.imshow(np.reshape(new_vectors[3], map_a.shape)).show()
# px.line(x = map_a.raman_shift, y = new_base_vectors[4]).show()
# px.imshow(np.reshape(new_vectors[4], map_a.shape)).show()
# px.line(x = map_a.raman_shift, y = new_base_vectors[5]).show()
# px.imshow(np.reshape(new_vectors[5], map_a.shape)).show()
# px.line(x = map_a.raman_shift, y = new_base_vectors[6]).show()
# px.imshow(np.reshape(new_vectors[6], map_a.shape)).show()


In [ ]:
# For abstract

# [] Trzeba zrobić trochę większe napisy na osiach (większe i może trochę grubsze). 
# [X] Zamiast "10 BN" i "11 BN" lepiej dać "h10BN" i "epitaxial hBN" (ten drugi nie jest izotopowo czysty z 11B tylko jest to naturalny BN z mieszanką 10B i 11B). 
# [X] Poza tym w podpisie oznaczasz kolejne panele jako (a), (b), (c), a na samych panelach nie ma tych oznaczeń - trzeba je dodać.

import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar

tfs = 24
fs = 19
fig, axs = plt.subplots(1, 3, figsize=(17, 5))

# Plot for new_base_vectors[0]
axs[0].plot(map_a.raman_shift, new_base_vectors[0]/np.max(new_base_vectors[0]), label='h$^{10}$BN')
axs[0].plot(map_a.raman_shift, new_base_vectors[2]/np.max(new_base_vectors[0]), label='epitaxial\nhBN')
axs[0].set_title('(a) PCA obtained spectra', fontsize=tfs, fontweight='bold')
axs[0].set_ylabel('Intensity [ab. units]', fontsize=fs)
axs[0].set_xlabel(r'Raman shift [$\text{cm}^{-1}$]', fontsize=fs)
axs[0].legend(fontsize=18, loc='upper left')

tmp = np.reshape(new_vectors[0], map_a.shape)
tmp[tmp < 0] = 0
tmp/=np.max(tmp)
im = axs[1].imshow(tmp, extent=[0,10,0,10], aspect=1)
# cax = fig.add_axes([axs[1].get_position().x1+0.01,axs[1].get_position().y0,0.02,axs[1].get_position().height])
cbar = fig.colorbar(im, ax=axs[1])#, orientation='vertical', fraction=0.046, pad=0.04)
cbar.set_label('Intensity [ab. units]', fontsize=fs)
cbar.ax.tick_params(labelsize=16)
axs[1].set_title('(b) h$\mathbf{^{10}}$BN', fontsize=tfs, fontweight='bold')
axs[1].set_xlabel('x [$\mu$m]', fontsize=fs)
axs[1].set_ylabel('y [$\mu$m]', fontsize=fs)

tmp = np.reshape(new_vectors[2], map_a.shape)
tmp[tmp < 0] = 0
tmp/=np.max(tmp)
im = axs[2].imshow(tmp, extent=[0,10,0,10], aspect=1)
# cax = fig.add_axes([axs[2].get_position().x1+0.01,axs[2].get_position().y0,0.02,axs[2].get_position().height])
cbar = fig.colorbar(im, ax=axs[2])#, orientation='vertical', fraction=0.046, pad=0.04)
cbar.set_label('Intensity [ab. units]', fontsize=fs)
cbar.ax.tick_params(labelsize=16)
axs[2].set_title(r'(c) epitaxial hBN', fontsize=tfs, fontweight='bold')
axs[2].set_xlabel('x [$\mu$m]', fontsize=fs)
axs[2].set_ylabel('y [$\mu$m]', fontsize=fs)

for i in range(3):
    axs[i].tick_params(axis='both', which='major', labelsize=17)

plt.tight_layout()
plt.savefig('img/abstract.png')
plt.show()

# px.line(x = map_a.raman_shift, y = new_base_vectors[0])
# tmp = np.reshape(new_vectors[0], map_a.shape)
# tmp[tmp < 0] = 0
# px.imshow(tmp).show()
# 
# px.line(x = map_a.raman_shift, y = new_base_vectors[1]).show()
# tmp = np.reshape(new_vectors[1], map_a.shape)
# tmp[tmp < 0] = 0
# px.imshow(tmp).show()
# 
# px.line(x = map_a.raman_shift, y = new_base_vectors[2]).show()
# tmp = np.reshape(new_vectors[2], map_a.shape)
# tmp[tmp < 0] = 0
# px.imshow(tmp).show()


In [ ]:
plotly_jinja_data = {}
for i in range(7):
    plotly_jinja_data["a_base_" + str(i)] = px.line(x = map_a.raman_shift, y = new_base_vectors[i]).update_layout(
    autosize=False,
    width=700,
    height=500).to_html(full_html=False)
    plotly_jinja_data["a_map_" + str(i)] = px.imshow(np.reshape(new_vectors[i], map_a.shape)).update_layout(
    autosize=False,
    width=500,
    height=500).to_html(full_html=False)

for i in range(7):
    plotly_jinja_data["b_base_" + str(i)] = "{{b_base_" + str(i) + "}}"
    plotly_jinja_data["b_map_" + str(i)] = "{{b_map_" + str(i) + "}}"

for i in range(8):
    plotly_jinja_data["c_base_" + str(i)] = "{{c_base_" + str(i) + "}}"
    plotly_jinja_data["c_map_" + str(i)] = "{{c_map_" + str(i) + "}}"

with open(output, "w", encoding="utf-8") as output_file:
    with open(input) as template_file:
        j2_template = Template(template_file.read())
        output_file.write(j2_template.render(plotly_jinja_data))


In [ ]:
map_a.plotSpectrum(map_a.posToId(0, 0))

In [ ]:
px.scatter(x = new_vectors[3], y = new_vectors[4]).show()
px.scatter(x = new_vectors[4], y = new_vectors[5]).show()
px.scatter(x = new_vectors[3], y = new_vectors[5]).show()

In [ ]:
np.dot(map_a.vectors.T, map_a.vectors)

In [ ]:
map_a.plotBaseVectors(5)
r = np.dot(new_vectors.T, new_base_vectors) 
# d = map_a.data_matrix - r
i = map_a.posToId(5, 30)
f = map_a.plotSpectrum(i)
f.add_scatter(x=map_a.raman_shift, y=r[i], mode='lines', name='Rekonstrukcja')
f.add_scatter(x=map_a.raman_shift, y=new_vectors[0][i] * new_base_vectors[0], mode='lines')
f.add_scatter(x=map_a.raman_shift, y=new_vectors[1][i] * new_base_vectors[1], mode='lines')
f.add_scatter(x=map_a.raman_shift, y=new_vectors[2][i] * new_base_vectors[2], mode='lines')
f.add_scatter(x=map_a.raman_shift, y=new_vectors[3][i] * new_base_vectors[3], mode='lines')

              
# f.add_scatter(x=map_a.raman_shift, y=d[i], mode='lines', name='Roznica')
f.update_layout(hovermode="x")
f.show()